<a href="https://colab.research.google.com/github/deshmukh2409/Artificial_Intelligence_Lab_SE_A_12/blob/master/exp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Minimax Algorithm
Minimax is a decision-making algorithm, typically used in game theory and artificial intelligence for two-player zero-sum games (meaning that one player's gain is equivalent to the other's loss). It's a recursive algorithm that aims to find the optimal move for a player, assuming that the opponent also plays optimally.

The algorithm works by building a game tree, where each node represents a state of the game, and edges represent possible moves. It then assigns a value to each node:
- For a **Maximizing Player** (whose turn it is), it chooses the move that leads to the highest possible score.
- For a **Minimizing Player** (the opponent), it chooses the move that leads to the lowest possible score for the Maximizing Player.

Let's implement Minimax using a simple game tree where leaf nodes have predefined utility values.

### Game Tree Representation
We'll represent our game tree as a dictionary where keys are nodes and values are either a list of child nodes (for non-leaf nodes) or an integer (for leaf nodes, representing their utility score). We'll assume player 'MAX' wants to maximize the score, and player 'MIN' wants to minimize it.

In [6]:
game_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': 3,
    'E': 12,
    'F': 8,
    'G': 2
}

# Helper function to check if a node is a leaf node
def is_leaf(node):
    return not isinstance(game_tree[node], list)

print("Defined Game Tree:")
for node, children_or_value in game_tree.items():
    print(f"{node}: {children_or_value}")

Defined Game Tree:
A: ['B', 'C']
B: ['D', 'E']
C: ['F', 'G']
D: 3
E: 12
F: 8
G: 2


### Minimax Algorithm Implementation
Now, let's implement the core Minimax function. It will recursively explore the game tree, alternating between maximizing and minimizing layers, until it reaches a leaf node (terminal state) and then propagates the values back up the tree.

In [7]:
def minimax(node, depth, maximizing_player):
    # If node is a leaf node or depth is 0 (though our tree is fixed depth)
    if is_leaf(node):
        return game_tree[node]

    if maximizing_player:
        max_eval = -float('inf')
        for child in game_tree[node]:
            # Recursively call minimax for children (now minimizing player's turn)
            eval = minimax(child, depth + 1, False)
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for child in game_tree[node]:
            # Recursively call minimax for children (now maximizing player's turn)
            eval = minimax(child, depth + 1, True)
            min_eval = min(min_eval, eval)
        return min_eval

# Demonstrate the Minimax algorithm starting from the root 'A'
# We assume 'A' is a maximizing player's turn, and initial depth is 0.
optimal_value = minimax('A', 0, True)
print(f"\nOptimal value for the maximizing player at the root ('A'): {optimal_value}")

# To find the best next move, we can iterate through the children of the root
# and see which one leads to the optimal value.
print("Determining the best next move:")
best_move = None
if 'A' in game_tree and isinstance(game_tree['A'], list):
    max_value_found = -float('inf')
    for child_move in game_tree['A']:
        value_of_move = minimax(child_move, 1, False) # Next player is MIN
        print(f"  Move to '{child_move}' leads to value: {value_of_move}")
        if value_of_move > max_value_found:
            max_value_found = value_of_move
            best_move = child_move

if best_move:
    print(f"The best next move for the maximizing player from 'A' is to node: {best_move}")
else:
    print("Could not determine a best move from 'A'.")


Optimal value for the maximizing player at the root ('A'): 3
Determining the best next move:
  Move to 'B' leads to value: 3
  Move to 'C' leads to value: 2
The best next move for the maximizing player from 'A' is to node: B


### A* Search Algorithm
The A* search algorithm is an informed search algorithm, meaning it uses heuristic information to guide its search. It is widely used in pathfinding and graph traversal. A* combines features of Dijkstra's algorithm and Greedy Best-First Search to find the shortest path from a start node to a goal node.

It evaluates nodes using a function $f(n) = g(n) + h(n)$, where:
- $g(n)$ is the cost from the start node to node $n$.
- $h(n)$ is the estimated cost from node $n$ to the goal node (heuristic function).

Let's define a sample graph with associated costs for edges and a heuristic function for each node to the goal.

In [4]:
# Define the graph with edge weights
graph_astar = {
    'A': {'B': 1, 'C': 3},
    'B': {'A': 1, 'D': 3, 'E': 6},
    'C': {'A': 3, 'F': 2},
    'D': {'B': 3, 'G': 2},
    'E': {'B': 6, 'G': 1},
    'F': {'C': 2, 'G': 4},
    'G': {}
}

# Define the heuristic function (estimated cost from each node to the goal 'G')
heuristics = {
    'A': 7,
    'B': 6,
    'C': 2,
    'D': 1,
    'E': 1,
    'F': 2,
    'G': 0
}

print("A* Graph with Edge Weights:")
for node, neighbors in graph_astar.items():
    print(f"{node}: {neighbors}")
print("\nHeuristics (to goal 'G'):")
for node, h_cost in heuristics.items():
    print(f"{node}: {h_cost}")

A* Graph with Edge Weights:
A: {'B': 1, 'C': 3}
B: {'A': 1, 'D': 3, 'E': 6}
C: {'A': 3, 'F': 2}
D: {'B': 3, 'G': 2}
E: {'B': 6, 'G': 1}
F: {'C': 2, 'G': 4}
G: {}

Heuristics (to goal 'G'):
A: 7
B: 6
C: 2
D: 1
E: 1
F: 2
G: 0


### A* Algorithm Implementation
We'll use a `priority queue` to store nodes to visit, prioritizing nodes with the lowest $f(n)$ value. We'll also keep track of the cost from the start node ($g(n)$) and the parent of each node to reconstruct the path.

In [5]:
import heapq

def astar_search(graph, heuristics, start, goal):
    # Priority queue stores (f_cost, node, path)
    priority_queue = [(heuristics[start], start, [start])]

    # Dictionary to store the cost from start to current node (g_cost)
    g_costs = {node: float('inf') for node in graph}
    g_costs[start] = 0

    # Set to keep track of visited nodes
    visited = set()

    while priority_queue:
        f_cost, current_node, path = heapq.heappop(priority_queue)

        if current_node in visited:
            continue

        visited.add(current_node)

        if current_node == goal:
            return path, g_costs[current_node] # Return path and total cost

        for neighbor, weight in graph[current_node].items():
            tentative_g_cost = g_costs[current_node] + weight

            if tentative_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g_cost
                f_cost = tentative_g_cost + heuristics[neighbor]
                heapq.heappush(priority_queue, (f_cost, neighbor, path + [neighbor]))

    return None, float('inf') # No path found

# Demonstrate A* search
start_node_astar = 'A'
goal_node_astar = 'G'

path, cost = astar_search(graph_astar, heuristics, start_node_astar, goal_node_astar)

if path:
    print(f"\nShortest path from '{start_node_astar}' to '{goal_node_astar}': {path}")
    print(f"Total cost: {cost}")
else:
    print(f"\nNo path found from '{start_node_astar}' to '{goal_node_astar}'.")


Shortest path from 'A' to 'G': ['A', 'B', 'D', 'G']
Total cost: 6


### Graph Representation
First, let's define a sample graph that we'll use for both BFS and DFS. We'll represent the graph using an adjacency list, where keys are nodes and values are lists of their neighbors.

In [1]:
graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

print("Sample Graph:")
for node, neighbors in graph.items():
    print(f"{node}: {neighbors}")

Sample Graph:
A: ['B', 'C']
B: ['A', 'D', 'E']
C: ['A', 'F']
D: ['B']
E: ['B', 'F']
F: ['C', 'E']


### Breadth-First Search (BFS)
BFS is an algorithm for traversing or searching tree or graph data structures. It starts at the tree root (or some arbitrary node of a graph, sometimes referred to as a 'search key') and explores all of the neighbor nodes at the present depth prior to moving on to the nodes at the next depth level.

It typically uses a queue data structure to keep track of the nodes to visit.

In [2]:
from collections import deque

def bfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    queue = deque([start_node])  # Initialize a queue with the starting node
    bfs_path = []

    visited.add(start_node)

    while queue:
        current_node = queue.popleft()  # Dequeue a node
        bfs_path.append(current_node)

        for neighbor in graph[current_node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)  # Enqueue unvisited neighbors
    return bfs_path

start_node_bfs = 'A'
bfs_result = bfs(graph, start_node_bfs)
print(f"BFS traversal starting from node '{start_node_bfs}': {bfs_result}")

BFS traversal starting from node 'A': ['A', 'B', 'C', 'D', 'E', 'F']


### Depth-First Search (DFS)
DFS is another algorithm for traversing or searching tree or graph data structures. It starts at the root (or an arbitrary node) and explores as far as possible along each branch before backtracking.

It typically uses a stack data structure (or recursion, which uses the call stack) to keep track of the nodes to visit.

In [3]:
def dfs(graph, start_node):
    visited = set()  # To keep track of visited nodes
    stack = [start_node]  # Initialize a stack with the starting node
    dfs_path = []

    while stack:
        current_node = stack.pop()  # Pop a node from the stack

        if current_node not in visited:
            visited.add(current_node)
            dfs_path.append(current_node)

            # Push unvisited neighbors onto the stack
            # (Order might vary depending on how neighbors are added,
            # often reversed to process in a consistent order)
            for neighbor in reversed(graph[current_node]): # Reverse to process in 'natural' order
                if neighbor not in visited:
                    stack.append(neighbor)
    return dfs_path

start_node_dfs = 'A'
dfs_result = dfs(graph, start_node_dfs)
print(f"DFS traversal starting from node '{start_node_dfs}': {dfs_result}")

DFS traversal starting from node 'A': ['A', 'B', 'D', 'E', 'F', 'C']
